# Imports

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from geopy.geocoders import Nominatim

# Cálculo da latitude e longitude

In [4]:
lista_cidades = []
lista_latitudes = []
lista_longitudes = []

cidades = ['Barcelona','Montserrat','Costa Brava','Cadaqués','Figueres','Girona','Vic e Rupit','Olot','Cap de Creus','Besalú','Garrotxa']
pais = 'Espanha'

for cidade in cidades:
    
    geolocator = Nominatim(user_agent="carattaolivajulia@gmail.com")

    localizacao = geolocator.geocode(f"{cidade}, {pais}")

    # Verifica se a localização foi encontrada
    if localizacao:
        print(f"{cidade}: {(localizacao.latitude, localizacao.longitude)}")
        lista_cidades.append(cidade)
        lista_latitudes.append(localizacao.latitude)
        lista_longitudes.append(localizacao.longitude)
    else:
        print(f"Coordenadas de {cidade} não encontradas.")
        lista_cidades.append(cidade)
        lista_latitudes.append(None)
        lista_longitudes.append(None)

df_lat_long = pd.DataFrame({'Cidade': lista_cidades, 'Latitude': lista_latitudes, 'Longitude': lista_longitudes})

Barcelona: (41.3828939, 2.1774322)
Montserrat: (39.3576494, -0.6031)
Costa Brava: (41.7037675, 2.9415943)
Cadaqués: (42.2893484, 3.2752159)
Figueres: (42.2666314, 2.9638434)
Girona: (41.9793006, 2.8199439)
Vic e Rupit: (42.0348785, 2.4600068)
Olot: (42.1822177, 2.4890211)
Cap de Creus: (42.319456, 3.3223466)
Besalú: (42.1986443, 2.6959559)
Garrotxa: (42.173081100000005, 2.5177403227464943)


In [5]:
df_lat_long

,Cidade,Latitude,Longitude
0,Barcelona,41.382894,2.177432
1,Montserrat,39.357649,-0.603100
2,Costa Brava,41.703767,2.941594
3,Cadaqués,42.289348,3.275216
4,Figueres,42.266631,2.963843
5,Girona,41.979301,2.819944
6,Vic e Rupit,42.034878,2.460007
7,Olot,42.182218,2.489021
8,Cap de Creus,42.319456,3.322347
9,Besalú,42.198644,2.695956


In [6]:
df_lat_long.to_csv('df_lat_long_espanha.csv')

# Cálculo da distância no globo

In [7]:
import math

def distancias_globo(lat1, lon1, lat2, lon2):
    
    # Converte latitude e longitude de graus para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças entre os pontos
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Raio da Terra em km (pode mudar para milhas se necessário)
    R = 6371.0
    distance = R * c
    
    return distance

In [8]:
def calcular_distancias_em_df(df):
    
    distancias = []
    
    for i, origem in df.iterrows():
        
        for j, destino in df.iterrows():
            
            if i != j:
                distancia = distancias_globo(origem['Latitude'], origem['Longitude'], destino['Latitude'], destino['Longitude'])
                distancias.append({'Origem': origem['Cidade'], 'Destino': destino['Cidade'], 'Distancia_km': distancia})

    return pd.DataFrame(distancias)

In [9]:
df_distancias_globo = calcular_distancias_em_df(df_lat_long)

In [10]:
df_distancias_globo.sort_values(by='Distancia_km', ascending=True)

,Origem,Destino,Distancia_km
107,Garrotxa,Olot,2.575399
79,Olot,Garrotxa,2.575399
37,Cadaqués,Cap de Creus,5.121574
83,Cap de Creus,Cadaqués,5.121574
109,Garrotxa,Besalú,14.956160
...,...,...,...
41,Figueres,Montserrat,441.211195
12,Montserrat,Cadaqués,461.165075
31,Cadaqués,Montserrat,461.165075
81,Cap de Creus,Montserrat,466.280374


# Cálculo do tempo

In [11]:
df_distancias_globo['tempo_horas'] = round((df_distancias_globo['Distancia_km']/80),2)
df_distancias_globo['tempo_minutos'] = round((df_distancias_globo['Distancia_km']/80)*60,2)

df_distancias_globo

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos
0,Barcelona,Montserrat,325.849591,4.07,244.39
1,Barcelona,Costa Brava,72.921390,0.91,54.69
2,Barcelona,Cadaqués,135.757113,1.70,101.82
3,Barcelona,Figueres,117.908084,1.47,88.43
4,Barcelona,Girona,85.117740,1.06,63.84
...,...,...,...,...,...
105,Garrotxa,Girona,32.960717,0.41,24.72
106,Garrotxa,Vic e Rupit,16.088612,0.20,12.07
107,Garrotxa,Olot,2.575399,0.03,1.93
108,Garrotxa,Cap de Creus,68.200188,0.85,51.15


In [12]:
df_distancias_globo['menores_tempos'] = df_distancias_globo.sort_values(by=['Origem', 'tempo_minutos']).groupby('Origem').cumcount() + 1

In [13]:
df_distancias_globo[df_distancias_globo['Origem'] == 'Barcelona'].sort_values(by='tempo_minutos')

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos,menores_tempos
1,Barcelona,Costa Brava,72.921390,0.91,54.69,1
5,Barcelona,Vic e Rupit,76.197570,0.95,57.15,2
4,Barcelona,Girona,85.117740,1.06,63.84,3
9,Barcelona,Garrotxa,92.284814,1.15,69.21,4
6,Barcelona,Olot,92.559329,1.16,69.42,5
8,Barcelona,Besalú,100.377755,1.25,75.28,6
3,Barcelona,Figueres,117.908084,1.47,88.43,7
2,Barcelona,Cadaqués,135.757113,1.70,101.82,8
7,Barcelona,Cap de Creus,140.844599,1.76,105.63,9
0,Barcelona,Montserrat,325.849591,4.07,244.39,10


In [14]:
df_distancias_globo[df_distancias_globo['Origem'] == 'Montserrat'].sort_values(by='tempo_minutos')

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos,menores_tempos
10,Montserrat,Barcelona,325.849591,4.07,244.39,1
15,Montserrat,Vic e Rupit,394.033208,4.93,295.52,2
11,Montserrat,Costa Brava,397.179479,4.96,297.88,3
16,Montserrat,Olot,407.919691,5.10,305.94,4
19,Montserrat,Garrotxa,408.699665,5.11,306.52,5
14,Montserrat,Girona,410.209536,5.13,307.66,6
18,Montserrat,Besalú,420.595330,5.26,315.45,7
13,Montserrat,Figueres,441.211195,5.52,330.91,8
12,Montserrat,Cadaqués,461.165075,5.76,345.87,9
17,Montserrat,Cap de Creus,466.280374,5.83,349.71,10


# Força bruta: cálculo de melhor rota

In [15]:
import networkx as nx
import itertools

# Criar o grafo usando NetworkX
G = nx.Graph()

# Adicionar as cidades e as distâncias no grafo
for _, row in df_distancias_globo.iterrows():
    G.add_edge(row['Origem'], row['Destino'], weight=row['Distancia_km'])

# Função para calcular o custo total de uma rota
def calcular_custo_rota(G, rota):
    custo_total = 0
    for i in range(len(rota) - 1):
        custo_total += nx.shortest_path_length(G, source=rota[i], target=rota[i+1], weight='weight')
    # Retorna ao ponto inicial para fechar o ciclo
    custo_total += nx.shortest_path_length(G, source=rota[-1], target=rota[0], weight='weight')
    return custo_total

# Função para encontrar a menor rota que passa por todas as cidades
def menor_rota_tsp(G):
    cidades = list(G.nodes)
    menor_rota = None
    menor_distancia = float('inf')

    # Gerar todas as permutações possíveis das cidades
    for rota in itertools.permutations(cidades):
        distancia_atual = calcular_custo_rota(G, rota)
        if distancia_atual < menor_distancia:
            menor_distancia = distancia_atual
            menor_rota = rota

    return menor_rota, menor_distancia

In [36]:
# # Encontrar e imprimir a menor rota que passa por todas as cidades

# rota, distancia = menor_rota_tsp(G)
# print(f"A menor rota que passa por todas as cidades é: {rota}, com uma distância de {distancia}")

In [37]:
# Demora muito!!!!!!!!!! Necessário utilizar um algoritmo para otimizar

# Problema do Caixeiro Viajante (TSP): cálculo de melhor rota

In [17]:
from pulp import *

In [18]:
df_lat_long

,Cidade,Latitude,Longitude
0,Barcelona,41.382894,2.177432
1,Montserrat,39.357649,-0.603100
2,Costa Brava,41.703767,2.941594
3,Cadaqués,42.289348,3.275216
4,Figueres,42.266631,2.963843
5,Girona,41.979301,2.819944
6,Vic e Rupit,42.034878,2.460007
7,Olot,42.182218,2.489021
8,Cap de Creus,42.319456,3.322347
9,Besalú,42.198644,2.695956


In [19]:
# Criar uma matriz de distâncias
n = len(df_lat_long)
matriz_distancia_cidades = np.zeros((n, n))

# Preencher a matriz com as distâncias calculadas
for i in range(n):
    for j in range(n):
        if i != j:
            matriz_distancia_cidades[i][j] = distancias_globo(df_lat_long['Latitude'][i],
                                                              df_lat_long['Longitude'][i], 
                                                              df_lat_long['Latitude'][j], 
                                                              df_lat_long['Longitude'][j])

            cidade_i = df_lat_long['Cidade'][i]
            cidade_j = df_lat_long['Cidade'][j]

            print(f'\n>> Iteração ({i}, {j}): ({cidade_i}, {cidade_j}). Distância: {matriz_distancia_cidades[i][j]}')


>> Iteração (0, 1): (Barcelona, Montserrat). Distância: 325.84959102523743

>> Iteração (0, 2): (Barcelona, Costa Brava). Distância: 72.92139024769585

>> Iteração (0, 3): (Barcelona, Cadaqués). Distância: 135.75711278089133

>> Iteração (0, 4): (Barcelona, Figueres). Distância: 117.90808375476341

>> Iteração (0, 5): (Barcelona, Girona). Distância: 85.11774024414085

>> Iteração (0, 6): (Barcelona, Vic e Rupit). Distância: 76.19757018193998

>> Iteração (0, 7): (Barcelona, Olot). Distância: 92.55932943026524

>> Iteração (0, 8): (Barcelona, Cap de Creus). Distância: 140.84459858676843

>> Iteração (0, 9): (Barcelona, Besalú). Distância: 100.37775464944617

>> Iteração (0, 10): (Barcelona, Garrotxa). Distância: 92.28481384831142

>> Iteração (1, 0): (Montserrat, Barcelona). Distância: 325.84959102523743

>> Iteração (1, 2): (Montserrat, Costa Brava). Distância: 397.1794793805916

>> Iteração (1, 3): (Montserrat, Cadaqués). Distância: 461.1650750873129

>> Iteração (1, 4): (Montserrat,

In [20]:
# Exibir a matriz de distâncias
print(matriz_distancia_cidades)

[[  0.         325.84959103  72.92139025 135.75711278 117.90808375
   85.11774024  76.19757018  92.55932943 140.84459859 100.37775465
   92.28481385]
 [325.84959103   0.         397.17947938 461.16507509 441.2111948
  410.20953622 394.03320848 407.91969132 466.28037355 420.59533025
  408.69966479]
 [ 72.92139025 397.17947938   0.          70.70970958  62.61461977
   32.25266043  54.27447006  65.04959017  75.34253113  58.65741124
   62.86798549]
 [135.75711278 461.16507509  70.70970958   0.          25.74145234
   50.96977749  72.90652591  65.81192454   5.12157424  48.73746167
   63.69093444]
 [117.90808375 441.2111948   62.61461977  25.74145234   0.
   34.08273575  48.87986566  40.20862435  30.06702143  23.31511898
   38.17996443]
 [ 85.11774024 410.20953622  32.25266043  50.96977749  34.08273575
    0.          30.3750005   35.42565044  56.08959357  26.44893467
   32.96071684]
 [ 76.19757018 394.03320848  54.27447006  72.90652591  48.87986566
   30.3750005    0.          16.55728113  

In [21]:
matriz_distancia_cidades.shape

(11, 11)

In [22]:
n = len(matriz_distancia_cidades)

# Criar loop
rotas_individuais = [(i,j) for i in range(n) for j in range(n) if matriz_distancia_cidades[i,j] != 0.0]

print(rotas_individuais)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9)]


In [23]:
# Aplicar o caixeiro viajante para minimização da distância total da rota
tsp = LpProblem("RotaCidadesEspanha", LpMinimize)

In [24]:
# Problema de Otimização
tsp

RotaCidadesEspanha:
MINIMIZE
None
VARIABLES

In [25]:
# Variáveis de decisão

## Variáveis xij: Binárias e indicam se a rota entre as cidades i e j é incluída na solução.
## Variáveis ui: Contínuas e representam a ordem das cidades para ajudar a prevenir a formação de sub-ciclos.

x = LpVariable.dicts("x", rotas_individuais, cat='Binary')

u = LpVariable.dicts("u", [i for i in range(n)], lowBound=1, upBound=n, cat='Continuous') # Ordem que as rotas individuais são visitadas

In [26]:
# x

In [27]:
# u

In [28]:
# Função Objetivo: 
## Minimizar a soma total das distâncias entre as cidades, multiplicada pela variável binária que indica se a rota é parte da solução ou não
## Portanto, se uma rota é escolhida (x[i,j] = 1), a distância correspondente é adicionada à soma total

# Somar a função objetivo à função já existente no problema

tsp += lpSum([matriz_distancia_cidades[i,j] * x[i,j] for (i,j) in rotas_individuais])
# lpSum: Soma lista de expressões lineares

In [29]:
# Imprimir a função objetivo
print(">> Função Objetivo:\n")
print(tsp.objective)

>> Função Objetivo:

325.84959102523743*x_(0,_1) + 92.28481384831142*x_(0,_10) + 72.92139024769585*x_(0,_2) + 135.75711278089133*x_(0,_3) + 117.90808375476341*x_(0,_4) + 85.11774024414085*x_(0,_5) + 76.19757018193998*x_(0,_6) + 92.55932943026524*x_(0,_7) + 140.84459858676843*x_(0,_8) + 100.37775464944617*x_(0,_9) + 325.84959102523743*x_(1,_0) + 408.6996647943191*x_(1,_10) + 397.1794793805916*x_(1,_2) + 461.1650750873129*x_(1,_3) + 441.2111947962737*x_(1,_4) + 410.2095362234402*x_(1,_5) + 394.0332084845814*x_(1,_6) + 407.9196913217714*x_(1,_7) + 466.2803735524424*x_(1,_8) + 420.595330247397*x_(1,_9) + 92.28481384831142*x_(10,_0) + 408.6996647943191*x_(10,_1) + 62.86798549347136*x_(10,_2) + 63.6909344365378*x_(10,_3) + 38.17996443343128*x_(10,_4) + 32.96071684031254*x_(10,_5) + 16.088612290887934*x_(10,_6) + 2.575398872082858*x_(10,_7) + 68.2001875065139*x_(10,_8) + 14.95615995735821*x_(10,_9) + 72.92139024769585*x_(2,_0) + 397.1794793805916*x_(2,_1) + 62.86798549347136*x_(2,_10) + 70.70

In [30]:
# Restrição 1: Garantir que cada cidade seja visitada apenas uma vez
for j in range(n):
    tsp += lpSum([x[i,j] for (i,m) in rotas_individuais if m==j]) == 1

# Restrição 2: Garantir que cada cidade seja partida exatamente uma vez
for i in range(n):
    tsp += lpSum([x[i,j] for (m,j) in rotas_individuais if m==i]) == 1

# Restrição 3: Eliminar subciclos
for (i,j) in rotas_individuais:
    if i > 0 and i != j:
        tsp += u[i] - u[j] + n*x[i,j] <= n-1

In [31]:
# Resolver o modelo
resolver_modelo = tsp.solve()
print(f'Status do problema: {LpStatus[resolver_modelo]}')

Status do problema: Optimal


In [32]:
# Exibir variáveis
for var in tsp.variables():
    if var.varValue > 0  and var.name.startswith('x'):
        print(f'{var.name}')

# 0: Barcelona
# 2: Costa Brava
# 5: Girona
# 3: Cadaqués
# 8: Cap de Creus
# 4: Figueres
# 9: Besalú
# 10: Garrotxa     
# 7: Olot
# 6: Vic e Rupit
# 1: Montserrat

x_(0,_2)
x_(1,_0)
x_(10,_7)
x_(2,_5)
x_(3,_8)
x_(4,_9)
x_(5,_3)
x_(6,_1)
x_(7,_6)
x_(8,_4)
x_(9,_10)


In [33]:
print('Rota: Barcelona -> Costa Brava -> Girona -> Cadaqués -> Cap de Creus -> Figueres -> Besalú -> Garrotxa -> Olot -> Vic e Rupit -> Montserrat')

Rota: Barcelona -> Costa Brava -> Girona -> Cadaqués -> Cap de Creus -> Figueres -> Besalú -> Garrotxa -> Olot -> Vic e Rupit -> Montserrat


In [34]:
lista_cidades_ordenadas = ['Barcelona', 'Costa Brava', 'Girona', 'Cadaqués', 'Cap de Creus', 'Figueres', 'Besalú', 'Garrotxa', 'Olot', 'Vic e Rupit', 'Montserrat']

dfs = []

# Percorre a lista de cidades sequencialmente (de origem para destino)
for i in range(len(lista_cidades_ordenadas) - 1):
    
    cidade_origem = lista_cidades_ordenadas[i]
    cidade_destino = lista_cidades_ordenadas[i + 1]
    
    df_temp = df_distancias_globo[(df_distancias_globo['Origem'] == cidade_origem) & (df_distancias_globo['Destino'] == cidade_destino)]
    dfs.append(df_temp)

df_resultante = pd.concat(dfs, ignore_index=True)

display(df_resultante.drop(['menores_tempos'], axis=1))

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos
0,Barcelona,Costa Brava,72.921390,0.91,54.69
1,Costa Brava,Girona,32.252660,0.40,24.19
2,Girona,Cadaqués,50.969777,0.64,38.23
3,Cadaqués,Cap de Creus,5.121574,0.06,3.84
4,Cap de Creus,Figueres,30.067021,0.38,22.55
5,Figueres,Besalú,23.315119,0.29,17.49
6,Besalú,Garrotxa,14.956160,0.19,11.22
7,Garrotxa,Olot,2.575399,0.03,1.93
8,Olot,Vic e Rupit,16.557281,0.21,12.42
9,Vic e Rupit,Montserrat,394.033208,4.93,295.52
